In [21]:
import numpy as np
import math

In [23]:
# GLOBAL VARIABLES
# text of 80 chars -> 640 bits
# we encode encode each bit into 51200/640 = 80 bits

N = 40960 #max is 51200, 40960 = 64*640
TOTAL_NB_CODEWORDS = 640 
CODEWORD_SIZE = 64 # 64 is the closest power 2 to 80

# with CODEWORD_SIZE = 2^i
i = 6

# M is CODEWORD_SIZE * CODEWORD_SIZE, we want sqrt(EPSILON/CODEWORD_SIZE) = 1
EPSILON = CODEWORD_SIZE

In [38]:
def compute_M(j):
    M_1 = np.array([[1,1],[1,-1]])
    temp = M_1
    for l in range (j-1):
        up = np.concatenate((temp,temp), axis=1) 
        bottom = np.concatenate((temp,-temp), axis=1)
        temp = np.concatenate((up,bottom), axis=0)
    return temp

In [39]:
M=compute_M(i)
print(M)

[[ 1  1  1 ...  1  1  1]
 [ 1 -1  1 ... -1  1 -1]
 [ 1  1 -1 ...  1 -1 -1]
 ...
 [ 1 -1  1 ... -1  1 -1]
 [ 1  1 -1 ...  1 -1 -1]
 [ 1 -1 -1 ... -1 -1  1]]


In [40]:
def columnk_M(M,k):
    return M[:,k-1]


In [41]:
columnk_M(M,2)

array([ 1, -1,  1, -1,  1, -1,  1, -1,  1, -1,  1, -1,  1, -1,  1, -1,  1,
       -1,  1, -1,  1, -1,  1, -1,  1, -1,  1, -1,  1, -1,  1, -1,  1, -1,
        1, -1,  1, -1,  1, -1,  1, -1,  1, -1,  1, -1,  1, -1,  1, -1,  1,
       -1,  1, -1,  1, -1,  1, -1,  1, -1,  1, -1,  1, -1])

In [42]:
def encode_mk(M,k,sign):
    return sign*columnk_M(M,k)

In [52]:
def compute_U(M,Y,j):
    if j==0:
        return Y
    if j==1:
        return np.dot(M, Y)
    U_t = np.dot(compute_M(j-1),Y[0:int(CODEWORD_SIZE)/2])
    U_b = np.dot(compute_M(j-1),Y[int(CODEWORD_SIZE)/2:])
    return np.concatenate((U_t+U_b,U_t-U_b),axis=0)
    
    

In [53]:
compute_U(compute_M(i), encode_mk(compute_M(i), 3, -1), i)

TypeError: slice indices must be integers or None or have an __index__ method

In [11]:
def decode_U(U,i):
    Umax = U.max(axis=1)
    Umin = U.min(axis=1)
    k = np.where(-Umin > Umax, Umin, Umax)
    if U[k]<0:
        return -k
    else :
        return k

In [12]:
def text_to_chunks(textfile,CHUNK_SIZE):
    f=open(textfile,'rb')
    data = f.read()
    f.close()
    noOfBytes = len(data)
    noOfChunks= noOfBytes/CHUNK_SIZE
    if(noOfBytes%CHUNK_SIZE):
        noOfChunks+=1
        
    chunkVector = []
    for i in range(0, noOfBytes+1, CHUNK_SIZE):
        np.append(chunkVector,data[i:i+ CHUNK_SIZE])
    
    return chunkVector

In [13]:
def chunks_to_text(fileName,chunkVector):
    f = open(fileName, 'wb')
    for chunk in chunkVector:
        f.write(chunk)
        f.close()
    return fileName    

In [14]:
def chunk_to_codewords(chunk,CHUNK_SIZE,epsilon):
    i = CHUNK_SIZE-1
    n = np.power(2,i)
    M = compute_M(i)
    chunkValue = chunk.dot(2**np.arange(chunkValue.size)[::-1])
    k = chunkValue%2
    if ((chunkValue/2) > n):
        sign = -1
    else :
        sign = 1
    return encode_mk(M,k,epsilon,n,sign)       

In [24]:
def transmitter(filename):
    n=np.power(2,15)
    print(len(filename)*8)
    CHUNK_SIZE = math.ceil(n/(len(filename)*8))
    print(CHUNK_SIZE)
    chunks = text_to_chunks(filename,CHUNK_SIZE)
    message = []
    for i in range (len(chunks)):
        np.append(message,chunk_to_codewords(chunks[i],CHUNK_SIZE,n))
    return np.savetxt("message.txt", message, fmt="%s")
    

In [25]:
def receiver(filename):
    n=np.power(2,15)
    i = math.log2(n)
    CHUNK_SIZE = math.ceil(n/(len(filename)*8))
    chunks = text_to_chunks(filename,CHUNK_SIZE)
    M = compute_M(i)
    message = []
    for j in range (len(chunks)):
        U = compute_U(M,chunk[j],i)
        np.append(message,decode_U(U,j))
    return np.savetxt("received.txt", message, fmt="%s")
    
        

In [26]:
transmitter("array.txt")

72
456
